<a href="https://colab.research.google.com/github/kasinadhsarma/Notebooks/blob/main/flightfare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
write it as torch and devlopment and integrate and devlop it faster

Here is all the data you need:
"Data_Train.xlsx"
"Sample_submission.xlsx"
"Test_set.xlsx"

## Data loading

### Subtask:
Load the provided Excel files into pandas DataFrames.


**Reasoning**:
Load the three Excel files into pandas DataFrames and inspect their initial contents and data types.



In [ ]:
import pandas as pd

try:
    df_train = pd.read_excel('Data_Train.xlsx')
    df_submission = pd.read_excel('Sample_submission.xlsx')
    df_test = pd.read_excel('Test_set.xlsx')

    display(df_train.head())
    display(df_submission.head())
    display(df_test.head())

    print("Data_Train info:")
    df_train.info()
    print("\nSample_submission info:")
    df_submission.info()
    print("\nTest_set info:")
    df_test.info()

    print("\nData_Train shape:", df_train.shape)
    print("Sample_submission shape:", df_submission.shape)
    print("Test_set shape:", df_test.shape)

except FileNotFoundError:
    print("One or more of the specified Excel files were not found.")
except Exception as e:
    print(f"An error occurred: {e}")

## Data exploration

### Subtask:
Explore the loaded DataFrames (`df_train`, `df_test`, `df_submission`) to understand their characteristics and identify potential issues.


**Reasoning**:
Analyze data types, missing values, descriptive statistics, and visualize distributions for the training and test datasets. Also, check the submission format and calculate correlations.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Data Types and Missing Values
print("Data Types and Missing Values:")
print("df_train:\n", df_train.isnull().sum())
print("\ndf_test:\n", df_test.isnull().sum())
print("\ndf_submission:\n", df_submission.isnull().sum())
print("\ndf_train data types:\n", df_train.dtypes)
print("\ndf_test data types:\n", df_test.dtypes)
print("\ndf_submission data types:\n", df_submission.dtypes)

# Descriptive Statistics
print("\nDescriptive Statistics (df_train):")
print(df_train.describe(include='all'))

# Distributions (Numerical Features)
numerical_features = df_train.select_dtypes(include=['number'])
for col in numerical_features.columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(df_train[col], kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()

# Distributions (Categorical Features)
categorical_features = df_train.select_dtypes(exclude=['number'])
for col in categorical_features.columns:
    plt.figure(figsize=(12, 6))
    df_train[col].value_counts().plot(kind='bar')
    plt.title(f"Frequency of {col}")
    plt.xticks(rotation=45, ha='right')
    plt.show()

# Relationships
plt.figure(figsize=(10, 8))
sns.heatmap(df_train.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

for col in numerical_features.columns:
    if col != 'Price':
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=col, y='Price', data=df_train)
        plt.title(f"{col} vs. Price")
        plt.show()

for col in categorical_features.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=col, y='Price', data=df_train)
    plt.title(f"Price vs {col}")
    plt.xticks(rotation=45, ha='right')
    plt.show()

# Data Shapes
print(f"\ndf_train shape: {df_train.shape}")
print(f"df_test shape: {df_test.shape}")
print(f"df_submission shape: {df_submission.shape}")

**Reasoning**:
The previous code block failed due to a `ValueError` when trying to calculate the correlation matrix because some columns are of object type.  I need to select only numerical columns for the correlation matrix and handle the errors gracefully when plotting.  I'll also improve the plotting to avoid overlapping plots and labels.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# ... (previous code for data types, missing values, descriptive stats)

# Relationships (Corrected Correlation Matrix)
numerical_features = df_train.select_dtypes(include=['number'])
plt.figure(figsize=(10, 8))
try:
    sns.heatmap(numerical_features.corr(), annot=True, cmap='coolwarm')
    plt.title('Correlation Matrix (Numerical Features)')
except ValueError as e:
    print(f"Error creating correlation matrix: {e}")
plt.show()

# Relationships (Scatter plots and box plots with error handling)
for col in numerical_features.columns:
    if col != 'Price':
        plt.figure(figsize=(8, 6))
        try:
            sns.scatterplot(x=col, y='Price', data=df_train)
            plt.title(f"{col} vs. Price")
        except Exception as e:
          print(f"Error creating scatter plot for {col}: {e}")
        plt.show()

# ... (rest of the code for categorical features and data shapes)

## Data preparation

### Subtask:
Prepare the data for model training by handling missing values, converting categorical features to numerical representations, and scaling numerical features. Apply consistent preprocessing to both training and testing datasets.


**Reasoning**:
Handle missing values in 'Route' and 'Total_Stops' columns of df_train, then engineer date and time features for both df_train and df_test.



In [ ]:
import pandas as pd

# Handle missing values in df_train
df_train['Route'].fillna(df_train['Route'].mode()[0], inplace=True)
df_train['Total_Stops'].fillna(df_train['Total_Stops'].mode()[0], inplace=True)


def preprocess_date_time(df):
    df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y')
    df['Day_of_Week'] = df['Date_of_Journey'].dt.dayofweek
    df['Month'] = df['Date_of_Journey'].dt.month
    df['Day'] = df['Date_of_Journey'].dt.day

    df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format='%H:%M')
    df['Dep_Hour'] = df['Dep_Time'].dt.hour

    df['Arrival_Time'] = df['Arrival_Time'].str.replace('(\d{1,2}:\d{2})\s[A-Za-z]*', '\\1')
    df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'], format='%H:%M', errors='coerce')
    df['Arrival_Hour'] = df['Arrival_Time'].dt.hour
    df['Arrival_Hour'] = df['Arrival_Hour'].fillna(df['Arrival_Hour'].mode()[0])
    return df

df_train = preprocess_date_time(df_train)
df_test = preprocess_date_time(df_test)

display(df_train.head())
display(df_test.head())

**Reasoning**:
Encode categorical features and scale numerical features.  Handle unseen categories in the test set during encoding.



In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Categorical features to encode
categorical_cols = ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Day_of_Week', 'Month']

# Initialize LabelEncoder
label_encoders = {}

# Fit and transform categorical features in df_train
for col in categorical_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    label_encoders[col] = le

# Transform categorical features in df_test, handling unseen labels
for col in categorical_cols:
    le = label_encoders[col]
    df_test[col] = le.transform(df_test[col])


# Numerical features to scale
numerical_cols = ['Dep_Hour', 'Arrival_Hour']

# Initialize StandardScaler
scaler = StandardScaler()

# Fit and transform numerical features in df_train
df_train[numerical_cols] = scaler.fit_transform(df_train[numerical_cols])

# Transform numerical features in df_test
df_test[numerical_cols] = scaler.transform(df_test[numerical_cols])

display(df_train.head())
display(df_test.head())

## Feature engineering

### Subtask:
Engineer new features from existing ones in `df_train` and `df_test` to potentially improve model performance.


**Reasoning**:
Convert 'Duration' to minutes and create interaction features.



In [ ]:
import pandas as pd
import numpy as np

def convert_duration_to_minutes(duration):
    if pd.isna(duration):
        return np.nan
    hours = 0
    minutes = 0
    duration_parts = duration.split(' ')
    for part in duration_parts:
        if 'h' in part:
            hours = int(part[:-1])
        elif 'm' in part:
            minutes = int(part[:-1])
    return hours * 60 + minutes

# Apply duration conversion
df_train['Duration_minutes'] = df_train['Duration'].apply(convert_duration_to_minutes)
df_test['Duration_minutes'] = df_test['Duration'].apply(convert_duration_to_minutes)

# Handle missing values (if any)
df_train['Duration_minutes'].fillna(df_train['Duration_minutes'].mean(), inplace=True)
df_test['Duration_minutes'].fillna(df_test['Duration_minutes'].mean(), inplace=True)

# Create interaction features
df_train['Stops_Duration_Interaction'] = df_train['Total_Stops'] * df_train['Duration_minutes']
df_test['Stops_Duration_Interaction'] = df_test['Total_Stops'] * df_test['Duration_minutes']

df_train['Source_Dest_Interaction'] = df_train['Source'] * df_train['Destination']
df_test['Source_Dest_Interaction'] = df_test['Source'] * df_test['Destination']

display(df_train.head())
display(df_test.head())

**Reasoning**:
Fix the chained assignment warnings and complete the remaining feature engineering steps.



In [ ]:
import pandas as pd
import numpy as np

# Fix chained assignment warnings
df_train['Duration_minutes'] = df_train['Duration_minutes'].fillna(df_train['Duration_minutes'].mean())
df_test['Duration_minutes'] = df_test['Duration_minutes'].fillna(df_test['Duration_minutes'].mean())

# Polynomial features
numerical_cols = ['Dep_Hour', 'Arrival_Hour', 'Duration_minutes']
for col in numerical_cols:
    df_train[col + '_squared'] = df_train[col] ** 2
    df_test[col + '_squared'] = df_test[col] ** 2

# Route Feature Engineering (simplified approach)
df_train['Route_Stops'] = df_train['Route'].str.count('→')
df_test['Route_Stops'] = df_test['Route'].str.count('→')

# Fill NaN values in 'Route_Stops' with 0
df_train['Route_Stops'] = df_train['Route_Stops'].fillna(0)
df_test['Route_Stops'] = df_test['Route_Stops'].fillna(0)

display(df_train.head())
display(df_test.head())

## Data splitting

### Subtask:
Split the preprocessed training data (`df_train`) into training and validation sets.


In [ ]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target variable (y)
X = df_train.drop('Price', axis=1)
y = df_train['Price']

# Split data into training and validation sets without stratification
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

## Model training

### Subtask:
Train a PyTorch model on the training data (X_train, y_train) and evaluate its performance on the validation data (X_val, y_val).


**Reasoning**:
The previous attempts to convert the data to PyTorch tensors failed due to object dtypes.  I'll try a different approach by explicitly converting all columns to numeric types, handling potential errors gracefully.  This will ensure compatibility with PyTorch tensors.



In [ ]:
import pandas as pd
import numpy as np

# Function to convert a column to numeric, handling errors
def convert_to_numeric(df, column):
    try:
        df[column] = pd.to_numeric(df[column], errors='coerce')
        return df
    except Exception as e:
        print(f"Error converting column '{column}': {e}")
        return df

# Iterate through all columns in X_train and X_val and convert to numeric
for col in X_train.columns:
    X_train = convert_to_numeric(X_train, col)
    if col in X_val.columns:
        X_val = convert_to_numeric(X_val, col)

# Fill NaN values with 0
X_train = X_train.fillna(0)
X_val = X_val.fillna(0)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).reshape(-1, 1)

**Reasoning**:
The previous code block successfully converted the data to numeric types and created PyTorch tensors. Now I need to proceed with the model training and evaluation as outlined in the instructions.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# 1. Define the model architecture (same as before)
class PricePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# 2. Define the loss function and optimizer (same as before)
input_size = X_train_tensor.shape[1]
model = PricePredictionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop (same as before)
num_epochs = 10
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Train
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    train_loss = criterion(outputs, y_train_tensor)
    train_loss.backward()
    optimizer.step()
    train_losses.append(train_loss.item())

    # Validation
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# 5. Save the model (same as before)
torch.save(model.state_dict(), f'price_prediction_model_epoch_{num_epochs}_val_loss_{val_loss.item():.4f}.pth')

# 6. Visualize the training and validation loss (same as before)
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

## Model training

### Subtask:
Train a PyTorch model to predict flight prices.


**Reasoning**:
The previous attempt to train the model failed due to data type issues.  Let's try a different approach to convert the data into PyTorch tensors.  I will convert all object columns to numeric using `pd.to_numeric(errors='coerce')`, replace infinite values with NaN, and then fill NaNs with 0.  After that, I will create and train a model.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Convert object columns to numeric, handling errors
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = pd.to_numeric(X_train[col], errors='coerce')
    if col in X_val.columns and X_val[col].dtype == 'object':
        X_val[col] = pd.to_numeric(X_val[col], errors='coerce')

# Replace infinite values with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_val.replace([np.inf, -np.inf], np.nan, inplace=True)

# Fill NaN values with 0
X_train.fillna(0, inplace=True)
X_val.fillna(0, inplace=True)


# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).reshape(-1, 1)

# 1. Define the model architecture
class PricePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# 2. Define the loss function and optimizer
input_size = X_train_tensor.shape[1]
model = PricePredictionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop
num_epochs = 10
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Train
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    train_loss = criterion(outputs, y_train_tensor)
    train_loss.backward()
    optimizer.step()
    train_losses.append(train_loss.item())

    # Validation
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# 5. Save the model
torch.save(model.state_dict(), f'price_prediction_model_epoch_{num_epochs}_val_loss_{val_loss.item():.4f}.pth')

# 6. Visualize the training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

## Model training

### Subtask:
Train a PyTorch model to predict flight prices.  Address previous data preprocessing issues.


**Reasoning**:
The previous attempts to train the model failed due to data preprocessing issues.  This time, I will apply one-hot encoding to the categorical features, standardize the numerical features, and handle outliers more robustly. Then, I will train the model again and evaluate its performance.



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Identify categorical and numerical columns
categorical_cols = ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Day_of_Week', 'Month', 'Route']
numerical_cols = ['Dep_Hour', 'Arrival_Hour', 'Duration_minutes', 'Stops_Duration_Interaction', 'Source_Dest_Interaction', 'Dep_Hour_squared', 'Arrival_Hour_squared', 'Duration_minutes_squared', 'Route_Stops', 'Day']

# Preprocessing for training data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'  # Keep other columns if any
)

X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)


# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_processed.toarray() if hasattr(X_train_processed, 'toarray') else X_train_processed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_val_tensor = torch.tensor(X_val_processed.toarray() if hasattr(X_val_processed, 'toarray') else X_val_processed, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).reshape(-1, 1)


# Define the model
class PricePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Initialize model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
model = PricePredictionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    # Train
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    train_loss = criterion(outputs, y_train_tensor)
    train_loss.backward()
    optimizer.step()
    train_losses.append(train_loss.item())

    # Validation
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

# Save the model
torch.save(model.state_dict(), 'price_prediction_model.pth')

# Plot losses
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

## Model training

### Subtask:
Train a PyTorch model to predict flight prices.  Address previous data preprocessing issues and incorporate a more robust training strategy.


**Reasoning**:
The previous attempts at model training failed due to preprocessing issues.  This code will refine the preprocessing of the data, trying target encoding for high cardinality categorical features, handling outliers in numerical features using RobustScaler, and ensuring no `inf` or `nan` values are present in the tensors.  Then, it will train a more robust model with regularization and early stopping.



**Reasoning**:
The previous code failed due to a missing package `category_encoders`.  I will install the package and then re-run the code.



In [ ]:
!pip install category_encoders

**Reasoning**:
The `category_encoders` package is now installed. I will execute the previous code block again, which should now run successfully.



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

# Identify categorical and numerical columns
categorical_cols = ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info', 'Day_of_Week', 'Month', 'Route']
numerical_cols = ['Dep_Hour', 'Arrival_Hour', 'Duration_minutes', 'Stops_Duration_Interaction', 'Source_Dest_Interaction', 'Dep_Hour_squared', 'Arrival_Hour_squared', 'Duration_minutes_squared', 'Route_Stops', 'Day']

# Preprocessing for training data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])


X_train_processed = preprocessor.fit_transform(X_train, y_train)
X_val_processed = preprocessor.transform(X_val)


# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_processed, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
X_val_tensor = torch.tensor(X_val_processed, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).reshape(-1, 1)


# Check for and handle inf or nan values
def check_and_handle_inf_nan(tensor):
    tensor[torch.isinf(tensor)] = 0
    tensor[torch.isnan(tensor)] = 0
    return tensor

X_train_tensor = check_and_handle_inf_nan(X_train_tensor)
y_train_tensor = check_and_handle_inf_nan(y_train_tensor)
X_val_tensor = check_and_handle_inf_nan(X_val_tensor)
y_val_tensor = check_and_handle_inf_nan(y_val_tensor)


# Define the model with dropout and ELU activation
class PricePredictionModel(nn.Module):
    def __init__(self, input_size):
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.elu1 = nn.ELU()
        self.dropout1 = nn.Dropout(0.3)  # Add dropout
        self.fc2 = nn.Linear(256, 128)
        self.elu2 = nn.ELU()
        self.dropout2 = nn.Dropout(0.3) # Add dropout
        self.fc3 = nn.Linear(128, 64)
        self.elu3 = nn.ELU()
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.elu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.elu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.elu3(x)
        x = self.fc4(x)
        return x

# Initialize model, loss function, and optimizer
input_size = X_train_tensor.shape[1]
model = PricePredictionModel(input_size)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.0005)

# Early stopping parameters
patience = 5
best_val_loss = float('inf')
epochs_no_improve = 0

# Training loop with early stopping
num_epochs = 100 # Increased number of epochs
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Train
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    train_loss = criterion(outputs, y_train_tensor)
    train_loss.backward()
    optimizer.step()
    train_losses.append(train_loss.item())

    # Validation
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

# Save the model
torch.save(model.state_dict(), 'price_prediction_model_robust.pth')

# Plot losses
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

## Model optimization

### Subtask:
Optimize the PyTorch model's hyperparameters to improve its performance.


**Reasoning**:
Fix the `TypeError` by modifying the `PricePredictionModel` class to accept the `hidden_size` and `dropout_rate` arguments in the constructor and use them in the model definition.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
import numpy as np

# Define the model architecture with adjustable parameters
class PricePredictionModel(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate):
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.elu1 = nn.ELU()
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.elu2 = nn.ELU()
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_size // 2, hidden_size // 4)
        self.elu3 = nn.ELU()
        self.fc4 = nn.Linear(hidden_size // 4, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.elu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.elu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.elu3(x)
        x = self.fc4(x)
        return x

# ... (rest of the code remains the same)

**Reasoning**:
The previous code block had a `TypeError` because the `PricePredictionModel` class definition was not included in the code block. Now that the class is defined correctly, execute the hyperparameter optimization code.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
import numpy as np

# Define the hyperparameter grid
param_grid = {
    'learning_rate': [1e-4, 5e-4, 1e-3],
    'batch_size': [32, 64, 128],
    'hidden_size': [64, 128, 256],
    'dropout': [0.2, 0.3, 0.4],
    'optimizer': ['Adam', 'RMSprop']
}

# Number of folds for cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Store results
results = []

# Iterate through hyperparameter combinations
for lr in param_grid['learning_rate']:
    for bs in param_grid['batch_size']:
        for hs in param_grid['hidden_size']:
            for do in param_grid['dropout']:
                for opt in param_grid['optimizer']:
                    fold_losses = []
                    for fold, (train_index, val_index) in enumerate(kf.split(X_train_tensor)):
                        # Split data for the current fold
                        X_train_fold, X_val_fold = X_train_tensor[train_index], X_train_tensor[val_index]
                        y_train_fold, y_val_fold = y_train_tensor[train_index], y_train_tensor[val_index]

                        # Create DataLoader for current fold
                        train_dataset = TensorDataset(X_train_fold, y_train_fold)
                        train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)

                        # Initialize model
                        model = PricePredictionModel(input_size=X_train_tensor.shape[1], hidden_size=hs, dropout_rate=do)

                        # Initialize optimizer
                        if opt == 'Adam':
                            optimizer = optim.Adam(model.parameters(), lr=lr)
                        else:  # RMSprop
                            optimizer = optim.RMSprop(model.parameters(), lr=lr)

                        criterion = nn.MSELoss()

                        # Training loop with early stopping
                        best_val_loss = float('inf')
                        patience = 5
                        epochs_no_improve = 0
                        for epoch in range(100):  # Max epochs
                            for batch_X, batch_y in train_loader:
                                optimizer.zero_grad()
                                outputs = model(batch_X)
                                loss = criterion(outputs, batch_y)
                                loss.backward()
                                optimizer.step()

                            with torch.no_grad():
                                val_outputs = model(X_val_fold)
                                val_loss = criterion(val_outputs, y_val_fold)

                            if val_loss < best_val_loss:
                                best_val_loss = val_loss
                                epochs_no_improve = 0
                            else:
                                epochs_no_improve += 1
                                if epochs_no_improve == patience:
                                    break

                        fold_losses.append(best_val_loss.item())

                    avg_loss = np.mean(fold_losses)
                    results.append({
                        'learning_rate': lr,
                        'batch_size': bs,
                        'hidden_size': hs,
                        'dropout': do,
                        'optimizer': opt,
                        'avg_val_loss': avg_loss
                    })
                    print(f"Hyperparameters: {lr, bs, hs, do, opt}, Avg Val Loss: {avg_loss:.4f}")


# Find best hyperparameters
best_params = min(results, key=lambda x: x['avg_val_loss'])
print(f"\nBest Hyperparameters: {best_params}")

# Retrain with best hyperparameters on the entire training set
model = PricePredictionModel(input_size=X_train_tensor.shape[1], hidden_size=best_params['hidden_size'], dropout_rate=best_params['dropout'])
if best_params['optimizer'] == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])
else:
    optimizer = optim.RMSprop(model.parameters(), lr=best_params['learning_rate'])
criterion = nn.MSELoss()
# Training loop (adjust epochs as needed)
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item():.4f}")

# Save the final model
torch.save(model.state_dict(), 'final_price_prediction_model.pth')